In [1]:
import pandas as pd

In [ ]:
data = pd.read_csv('../datasets/car-sales-extended-missing-data.csv')

In [3]:
data.isnull().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

here we decided to have all the nulls in KM be filled with the mean average of others

In [4]:
data['Odometer (KM)'] = data['Odometer (KM)'].fillna(data['Odometer (KM)'].mean())

the first 2 tell us the number of records in our dataset, while the last one tells us the number of values in each column.

In [5]:
len(data), data.shape[0], data.count()

(1000,
 1000,
 Make              951
 Colour            950
 Odometer (KM)    1000
 Doors             950
 Price             950
 dtype: int64)

now we might want to just straight up drop the records with NaN values in a specific column

the code below wont work

In [6]:
# data['Colour'] = data['Colour'].dropna(data['Colour'].isnull())

we have two ways: we can either use assingment with notna. essentially we are replacing the before data with data that only has data that isnt NaN in the mentioned column

In [7]:
data = data[data['Colour'].notna()]

or we can just use dropna with subset that has the list of columns we want to apply the rule to

In [8]:
data.dropna(subset=['Colour', 'Doors'], inplace=True)

as you can see the amount of records has been reduced by 50 from colours and 50 from doors

In [9]:
len(data)

904

here we can convert the data from one datatype to another. forexample the datatype of Doors was float while each of them are int numbers. we can convert them using .astype() 

In [10]:
data['Doors'] = data['Doors'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 904 entries, 0 to 999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Make           858 non-null    object 
 1   Colour         904 non-null    object 
 2   Odometer (KM)  904 non-null    float64
 3   Doors          904 non-null    int64  
 4   Price          860 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 42.4+ KB


here we delve into the preprocessing section: normalization

it scales the data from unspecific numbers to a known and standard range. there are multiple methods to do this. one of them is using iqr

In [11]:
from sklearn.preprocessing import MinMaxScaler

lets first drop the null values and sort the dataset based on its price from highest to lowest

In [12]:
data = data[data['Price'].notna()]
data = data.sort_values(by='Price', ascending=False)

one way of normalization is using the MinMaxScaler() method. it scales the highest value to lowest value in a 1 to 0 scale as shown in the newly created column Scaled Price

In [13]:

sc = MinMaxScaler()
data[['Scaled Price']] = sc.fit_transform(data[['Price']])
data

,Make,Colour,Odometer (KM),Doors,Price,Scaled Price
906,BMW,White,16274.000000,5,52458.0,1.000000
720,BMW,White,57905.000000,3,50868.0,0.967984
55,BMW,White,79937.000000,5,48686.0,0.924047
978,BMW,White,85739.000000,5,48419.0,0.918670
272,BMW,White,23996.000000,5,48239.0,0.915046
...,...,...,...,...,...,...
406,Honda,Blue,210446.000000,4,3594.0,0.016069
86,Nissan,Blue,131253.237895,4,3300.0,0.010149
173,Nissan,Black,240536.000000,4,3247.0,0.009081
845,Toyota,Green,243177.000000,4,3125.0,0.006625


we can get the first quentile and the last quentile and of the dataset ( remember it has to be a pandas file).
then we get its iqr by subtracting them. these are all methamatics and standard practice that we will delve into deeper later on.

In [14]:
q1 = data['Price'].quantile(0.25)
q3 = data['Price'].quantile(0.75)
iqr = q3 - q1


lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr


now we filter the data with assingment with 2 conditions. we need both to be through hense we use 'and' in between them. remember that this is pandas syntax not pain python so we use & instead of 'and'.

In [15]:
data = data[(data['Price'] >= lower) & (data['Price'] <= upper)]


now a small part of the last quarter and first quarter are deleted from the file. we are essentially getting rid of the two extremes of the bell curve.

In [16]:
data

,Make,Colour,Odometer (KM),Doors,Price,Scaled Price
682,BMW,Black,54141.000000,5,37518.0,0.699166
837,BMW,Red,149413.000000,5,36864.0,0.685997
709,BMW,Blue,134103.000000,5,36109.0,0.670795
89,NaN,Blue,90249.000000,4,35988.0,0.668358
261,Toyota,White,61623.000000,4,35760.0,0.663767
...,...,...,...,...,...,...
406,Honda,Blue,210446.000000,4,3594.0,0.016069
86,Nissan,Blue,131253.237895,4,3300.0,0.010149
173,Nissan,Black,240536.000000,4,3247.0,0.009081
845,Toyota,Green,243177.000000,4,3125.0,0.006625
